In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os

In [39]:
tf.random.set_seed(8)


In [40]:
path = os.path.abspath('')
path = path + "/engineered_data_250ms_window80_step8.csv"
df = pd.read_csv(path)


diction = {'Dany':0, 'Felix':1, 'Julian':2, 'Mark':3,'Martin':4,'Michele':5,'Paul':6}

for index,row in df['Participant'].items():
    df.loc[index,'Participant'] = diction[row]

List = ["Window", "Participant", "Run", "Path"]
for name in df.columns:
    if "Acceleration" in name and not "Linear" in name:
        List.append(name)

# df.drop(df.columns.difference(List),axis=1,inplace=True)
df.columns

Index(['Window', 'Participant', 'Path', 'Run',
       'median_Acceleration x (m/s^2)_head_mean',
       'median_Acceleration x (m/s^2)_head_std',
       'median_Acceleration x (m/s^2)_head_last',
       'median_Acceleration x (m/s^2)_head_get_freq',
       'median_Acceleration x (m/s^2)_head_get_weighted_freq',
       'median_Acceleration x (m/s^2)_head_get_power_spectral_entropy',
       ...
       'median_Z (rad/s)_leg_last', 'median_Z (rad/s)_leg_get_freq',
       'median_Z (rad/s)_leg_get_weighted_freq',
       'median_Z (rad/s)_leg_get_power_spectral_entropy',
       'median_Z (µT)_leg_mean', 'median_Z (µT)_leg_std',
       'median_Z (µT)_leg_last', 'median_Z (µT)_leg_get_freq',
       'median_Z (µT)_leg_get_weighted_freq',
       'median_Z (µT)_leg_get_power_spectral_entropy'],
      dtype='object', length=226)

In [41]:
#TEST/TRAIN SPLIT GOES HERE
X_Train = df[df['Run'] < 4].copy()
X_Train
X_Test = df[df['Run'] == 4].copy()

Y_Train = X_Train['Participant'].copy().to_numpy()

Y_Test = X_Test['Participant'].copy().to_numpy()


X_Test.drop(['Participant'],axis=1,inplace=True)
X_Train.drop(['Participant'],axis=1,inplace=True)
X_Train.loc[X_Train['Path'] == 'circle', "Path"] = 1
X_Train
X_Train.loc[X_Train['Path'] == 'straight', "Path"] = 0
X_Test.loc[X_Test['Path'] == 'circle',"Path"] = 1
X_Train
X_Test.loc[X_Test['Path'] == 'straight',"Path"] = 0
X_Train
X_Train = X_Train.to_numpy()
X_Test = X_Test.to_numpy()
X_Train.shape
X_Train = np.asarray(X_Train).astype(np.float32)
X_Test = np.asarray(X_Test).astype(np.float32)
tf.convert_to_tensor(X_Test,dtype=tf.float32)
tf.convert_to_tensor(X_Train,dtype=tf.float32)

<tf.Tensor: shape=(1019, 225), dtype=float32, numpy=
array([[1.0000000e+00, 1.0000000e+00, 1.0000000e+00, ...,           nan,
                  nan,           nan],
       [2.0000000e+00, 1.0000000e+00, 1.0000000e+00, ...,           nan,
                  nan,           nan],
       [3.0000000e+00, 1.0000000e+00, 1.0000000e+00, ...,           nan,
                  nan,           nan],
       ...,
       [3.0000000e+01, 0.0000000e+00, 3.0000000e+00, ..., 2.5000000e-02,
        1.2790108e-01, 3.4416955e+00],
       [3.1000000e+01, 0.0000000e+00, 3.0000000e+00, ..., 0.0000000e+00,
                  nan,           nan],
       [3.2000000e+01, 0.0000000e+00, 3.0000000e+00, ..., 0.0000000e+00,
                  nan,           nan]], dtype=float32)>

In [42]:
#Reshape it
X_Train.reshape((X_Train.shape[0],1,X_Train.shape[1]))
X_Test.reshape((X_Test.shape[0],1,X_Test.shape[1]))

array([[[1.0000000e+00, 1.0000000e+00, 4.0000000e+00, ...,
                   nan,           nan,           nan]],

       [[2.0000000e+00, 1.0000000e+00, 4.0000000e+00, ...,
                   nan,           nan,           nan]],

       [[3.0000000e+00, 1.0000000e+00, 4.0000000e+00, ...,
                   nan,           nan,           nan]],

       ...,

       [[2.2000000e+01, 1.0000000e+00, 4.0000000e+00, ...,
         1.2500000e-02, 9.9776268e-02, 2.6692116e+00]],

       [[2.3000000e+01, 1.0000000e+00, 4.0000000e+00, ...,
         0.0000000e+00,           nan,           nan]],

       [[2.4000000e+01, 1.0000000e+00, 4.0000000e+00, ...,
         0.0000000e+00,           nan,           nan]]], dtype=float32)

In [43]:
# create and fit the LSTM network
look_back = 1
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_Train, Y_Train, epochs=100, batch_size=1, verbose=2)

/home/paul/Master/Quant/ML4QS-bike-user-recognition/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [25]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

input_size = 10
sequence_length = 10
num_layers = 2
hidden_size = 256

learning_rate = 0.001
num_epochs = 5
num_classes = 7
batch_size = 64


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SimpleGRU(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [26]:
model = SimpleGRU().to(device=device)
x = torch.randn(64,10,10).to(device=device)
y = model(x)
x.shape,y.shape

loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
y

tensor([[ 0.0327, -0.0634, -0.0058, -0.0321,  0.0628,  0.0862, -0.0098],
        [ 0.0188, -0.0644, -0.0378, -0.0357,  0.0343,  0.0577, -0.0523],
        [-0.0082, -0.0535, -0.0086, -0.0477,  0.0366,  0.0763, -0.0186],
        [-0.0045, -0.0285, -0.0314, -0.0491,  0.0479,  0.0511, -0.0032],
        [ 0.0300, -0.0262, -0.0380, -0.0406,  0.0396,  0.0581, -0.0122],
        [ 0.0510, -0.0452,  0.0055, -0.0245,  0.0568,  0.0450,  0.0015],
        [-0.0060, -0.0222, -0.0123, -0.0575,  0.0463,  0.0492, -0.0050],
        [ 0.0147, -0.0336, -0.0063, -0.0307,  0.0569,  0.0744, -0.0127],
        [-0.0215, -0.0475, -0.0221, -0.0310,  0.0614,  0.0626,  0.0038],
        [-0.0044, -0.0577, -0.0176, -0.0518,  0.0464,  0.0592, -0.0063],
        [ 0.0275, -0.0314, -0.0138, -0.0314,  0.0509,  0.0673, -0.0082],
        [ 0.0095, -0.0597, -0.0320, -0.0178,  0.0368,  0.0849,  0.0074],
        [-0.0119, -0.0347, -0.0374, -0.0381,  0.0228,  0.0377,  0.0117],
        [ 0.0098, -0.0313,  0.0037, -0.0029,  0.028

In [27]:
x[0]

tensor([[-0.1822,  0.5373, -0.3509,  0.6588, -0.3443,  0.3314,  0.6150, -0.3198,
         -1.2717, -0.5543],
        [-0.8845, -1.8585,  0.1797, -0.1284, -0.6218,  0.9519, -0.9108, -0.0652,
          0.3551,  0.2951],
        [-1.5271,  0.6252,  0.9545, -1.4229,  0.6776, -0.5026, -0.3389, -0.2303,
         -1.3512, -1.5314],
        [-0.5960,  0.8212,  0.8791, -0.0399,  1.0438,  0.4319, -1.0109, -1.8645,
         -0.3983,  1.2355],
        [ 0.2788, -0.0725, -0.1116,  0.0456,  0.8689, -1.0621,  0.3665,  0.3647,
         -0.2530,  0.7290],
        [-1.5337,  1.1668, -1.0408,  0.1020, -0.7847, -1.0911, -0.4360, -1.2183,
         -0.8046, -0.2357],
        [ 0.6160,  0.9218, -0.6874,  1.7300,  0.7430, -1.4280, -0.1399, -0.0365,
          0.7904, -0.8181],
        [-1.5431,  0.1405, -0.3673, -0.1808,  0.1712, -0.5992,  0.4093,  1.7636,
          0.1674, -0.1030],
        [-0.7666,  0.4063, -0.4273,  1.0159, -0.8186,  0.8651,  0.9734, -1.2224,
          1.1082, -0.7555],
        [-2.2189,  

In [28]:
x = torch.randn(batch_size, sequence_length, input_size).to(device)
y = torch.randint(0, num_classes, (batch_size,)).to(device)

In [29]:
current_loss = 0
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(x)
    loss = loss_criterion(outputs, y)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    print(f"At epoch: {epoch}, loss: {loss}")

At epoch: 0, loss: 1.9496396780014038
At epoch: 1, loss: 1.8942210674285889
At epoch: 2, loss: 1.8460768461227417
At epoch: 3, loss: 1.8008283376693726
At epoch: 4, loss: 1.7577062845230103
